In [0]:
df = spark.read.table("bikes.bronze.crm_sales_details")

Importing the libraries

In [0]:
from pyspark.sql.functions import col, trim, when, to_date
from pyspark.sql.types import StringType

triming the columns

In [0]:
for field in df.schema:
  if field.dataType == StringType():
    df = df.withColumn(field.name, trim(col(field.name)))

Fixing the numric null values and rename columns names

In [0]:
# clean fun
df_clean = (
    df
    # order date
    .withColumn(
        "sls_order_dt",
        when(col("sls_order_dt").isin(0, "0", "00000000"), None)
        .otherwise(to_date(col("sls_order_dt").cast("string"), "yyyyMMdd"))
    )

    # ship date
    .withColumn(
        "sls_ship_dt",
        when(col("sls_ship_dt").isin(0, "0", "00000000"), None)
        .otherwise(to_date(col("sls_ship_dt").cast("string"), "yyyyMMdd"))
    )

    # due date
    .withColumn(
        "sls_due_dt",
        when(col("sls_due_dt").isin(0, "0", "00000000"), None)
        .otherwise(to_date(col("sls_due_dt").cast("string"), "yyyyMMdd"))
    )

    # Rename
    .withColumnRenamed("sls_ord_num", "order_number")
    .withColumnRenamed("sls_prd_key", "product_key")
    .withColumnRenamed("sls_cust_id", "customer_id")
    .withColumnRenamed("sls_order_dt", "order_date")
    .withColumnRenamed("sls_ship_dt", "ship_date")
    .withColumnRenamed("sls_due_dt", "due_date")
    .withColumnRenamed("sls_sales", "total_sales")
    .withColumnRenamed("sls_quantity", "quantity")
    .withColumnRenamed("sls_price", "unit_price")
)

Save the table

In [0]:
df.write.mode("overwrite").saveAsTable("bikes.silver.crm_sales_details")